In [1]:
from collections import Counter
import math
from operator import itemgetter


def triplets(s: bytes):
    s = b' ' + s.strip() + b' '
    memview = memoryview(s)
    for i in range(len(s)-2):
        yield memview[i:i+3]
        

def all_triplets_from_many_lines(lines):
    for line in lines:
        yield from triplets(line)
        

def load_file(filename):
    with open(filename, 'rb') as file:
        return {line.rstrip(b'\r\n'): i for i, line in enumerate(file.readlines())}
    
    
def account_triplets(lines):
    c = Counter(all_triplets_from_many_lines(lines))
    m = max(c.values())
    for key in c:
        c[key] /= m  # Нормализуем по максимальному значению
    return c


def get_score(s: bytes):
    # return sum(c[t] for t in triplets(s)) / len(s)
    return sum(c[t] for t in triplets(s)) / math.log(len(s)+1)  # Преимущество длинным строкам

In [2]:
old_file = load_file('../stringdumps/stringdump_0_47_04.txt')
old_file.update(load_file('../stringdumps/stringdump_0_47_05.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_47_03.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_47_02.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_47_01.txt'))
old_file.update(load_file('../stringdumps/stringdump_0_44_12.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_0_50_01.txt'))
old_file.update(load_file('../stringdumps/stringdump_steam_0_50_02.txt'))

c = account_triplets(old_file)  # Обучаем на старых файлах

In [3]:
new_file = load_file('out2.txt')

In [4]:
diff = sorted(new_file.keys()-old_file.keys(), key=lambda s: get_score(s))

In [5]:
for item in diff:
    print(item, get_score(item))

b'\xe2\x95\xa8Kj' 0.0
b'\xe2\x95\xa1b\xe2\x89\xa1\xe2\x94\x82' 0.0
b'`\xc3\xa7j' 0.0
b'\xe2\x95\x96N\xe2\x95\xa3' 0.0
b'\xe2\x96\x91\xc3\xb4G' 0.0
b'a}z\xe2\x89\xa1\xe2\x94\x82' 0.0
b'0\xcf\x84m' 0.0
b'\xe2\x94\x82F' 0.0
b'\xc3\xb2\xe2\x94\xbcw\xe2\x89\xa1\xe2\x94\x82' 0.0
b'wf\xe2\x96\x92' 0.0
b'\xc2\xab\xe2\x96\x90n' 0.0
b'8b\xc2\xa3' 0.0
b'p"\xc3\xa8' 0.0
b'l4' 0.0
b'A\xe2\x95\xa3r\xe2\x89\xa1\xe2\x94\x82' 0.0
b'<\xe2\x96\x90V' 0.0
b'\xe2\x95\xa8W' 0.0
b'Wm\xc3\x91' 0.0
b'W\xc3\xa4u' 0.0
b'a#\xc3\xb2\xc3\xb9\xe2\x89\xa1\xe2\x94\x82' 0.0
b'i\xe2\x96\x91~' 0.0
b'P\xe2\x95\xa8' 0.0
b'z\xe2\x89\xa1\xe2\x94\x82' 0.0
b'aW\xe2\x95\x925\xe2\x89\xa1\xe2\x94\x82' 0.0
b'\xe2\x94\x82h8' 0.0
b'p\xc2\xa3\xe2\x88\x9e' 0.0
b"'t\xc3\xa2" 0.0
b'pK=' 0.0
b"'t\xc3\xab" 0.0
b'\xe2\x94\xa4F' 0.0
b'@\xc2\xb5S' 0.0
b'\xc2\xbba8' 0.0
b'\xe2\x82\xa7s9' 0.0
b'X\xc2\xa2\xc3\xbc' 0.0
b'-G' 0.0
b'\xce\xb1s\xc3\x91' 0.0
b'Z\xce\x98=' 0.0
b'SR' 0.0
b'`c\xc3\x89' 0.0
b"'tK" 0.0
b'd\xe2\x94\x82\xc3\x84' 0.0
b'r\xe2\

In [6]:
threshold = 0.046

In [7]:
with open('../stringdumps/stringdump_steam_0_50_05_sorted_by_xrefs.txt', 'wb') as output:
    for line, number in sorted(new_file.items(), key=itemgetter(1)):
        if line in diff and get_score(line) < threshold: # Отсеиваем только добавившиеся строки
            continue
        
        output.write(line)
        output.write(b'\n')